In [1]:
!pip install ket-lang
!pip install numpy
import numpy as np
import math
from ket import *

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


A Estimativa de Fase Quântica (QPE) é um dos algoritmos quânticos mais fundamentais, servindo como peça central para diversos outros algoritmos quânticos, incluindo o famoso algoritmo de Shor para fatoração e simulações quânticas. O QPE é a ponte que conecta operadores unitários com informação mensurável, permitindo-nos "ler" as fases dos autovetores - propriedades intrinsecamente quânticas que permanecem inacessíveis aos métodos clássicos.

# Estimativa de Fase Quântica (QPE)

**AVISO:** Para o entedimento completo do algorítmo de estimação de fase quântica é necessário conhecimento perante a Transformada Quântica de Fourier

O algoritmo opera em dois conjuntos de qubits, referidos neste contexto como **registros**. Os dois registros contêm $n$ e $m$ qubits, respectivamente. Seja $U$ um operador unitário atuando no registro de $m$ qubits.

Os autovalores de um operador unitário possuem **módulo unitário** e são, portanto, caracterizados por sua **fase**. Assim, se $\ket{\psi}$ é um autovetor de $U$, então:

$$
U\ket{\psi} = e^{2\pi i\theta}\ket{\psi}
$$

para algum $\theta \in \reals$. Devido à periodicidade da exponencial complexa, podemos sempre assumir que $0 \leq \theta < 1$.

**Objetivo**

O objetivo do algoritmo é produzir uma estimativa precisa para $\theta$ utilizando um número reduzido de portas quânticas e mantendo alta probabilidade de sucesso. A estimativa de fase quântica alcança esse desempenho assumindo acesso oráculo a $U$ e disponibilidade de $\ket{\psi}$ como estado quântico. Esta abordagem permite que nos concentremos no número de aplicações de $U$, sem precisar considerar o custo de implementação do operador em si.

O algoritmo retorna com alta probabilidade uma aproximação para $\theta$ com erro aditivo $\varepsilon$, utilizando $\text{n} \space = O(log(\frac{1}{\varepsilon}))$ qubits no primeiro registro e $O(\frac{1}{\varepsilon})$ operações controladas de $U$. A probabilidade de sucesso pode ser melhorada para $1 - \Delta$ para qualquer $\Delta > 0$ através do uso de $O(\frac{log(\frac{1}{\Delta})}{ε})$ aplicações de operações controladas por U, sendo este limite considerado ótimo para o algoritmo.


## Algoritmo Quântico de Estimação de Fase (QPE)

O Algoritmo Quântico de Estimação de Fase (QPE) é uma aplicação fundamental da Transformada Quântica de Fourier (QFT) que permite estimar o autovalor de um operador unitário.

Dado um operador unitário $U$ e um autovetor $\ket{\psi}$ tal que:

$$
U\ket{\psi} = e^{2\pi i\phi}\ket{\psi}
$$

O QPE estima a fase ${ \phi \in [0,1) }$.

### Circuito Quântico para o QPE

O algoritmo quântico para a Estimação de Fase Quântica é implementado através de um circuito composto por portas Hadamard e operadores unitários. O procedimento para $m$ qubits em ${\ket{0}}$ e $n$ qubits é:

**Procedimento:**

$$
\begin{array}{ccc}
   \text{Etapa 0:} & \ket{0}^{\otimes m} \ket{v} & \text{Inicializa m qubits em 0 e n qubits em v} \\
   \text{Etapa 1:} & H\ket{\psi_0} & \text{Coloca os qubits incializados em 0 em superposição} \\
   \text{Etapa 2:} & U^1\ket{\psi_1} & \text{Aplica o operador unitário no primeiro qubit v} \\
   \vdots & \vdots & \vdots \\
   \text{Etapa m:} & U^{2^{m-1}}\ket{\psi_{m-1}} & \text{Aplica o operador unitário no n-ésimo qubit em v} \\
   \text{Etapa m+1:} & \text{IQFT} \ket{\psi_m} & \text{Aplica a IQFT nos m qubits} \\
\end{array}
$$

**Circuito**

Notação expandida:

![PEA.webp](../images/algoritmos/qpe/PEA.webp)

Observação: Nessa imagem notamos uma diferença de notação para o procedimento, em que: "$m \space qubits$" se tornaram "$n \space qubits$" e "$IQFT$" se tornou "$QFT^\dagger$"

Análise detalhada do algoritmo:

**Estado inicial:** $\ket{0}^{\otimes m} \ket{v}$

Aplicamos Hadamard nos qubits inicializados em $\ket{0}$ (primeiro registrador):

$$
\frac{1}{\sqrt{2^t}} \sum_{j=0}^{2^t-1} \ket{j} \otimes \ket{v}
$$


Aplicação Controlada de U, Para cada qubit $k$ da primeira register $( k = 0, 1, ..., t-1 )$:

   - Aplicar porta U controlada $2^k$ vezes:

   $$
   CU^{2^k} \ket{k}\ket{v} = \ket{k} \otimes U^{2^k}\ket{v} = e^{2\pi i \phi 2^k} \ket{k}\ket{v}
   $$

   - Resultando em:

   $$
   \frac{1}{\sqrt{2^t}} \sum_{j=0}^{2^t-1} e^{2\pi i \phi j} \ket{j} \otimes \ket{v}
   $$

Aplicação a IQFT no primeiro registrador:

$$
   \ket{\tilde{\phi}} \otimes \ket{v}
$$

**Complexidade**

A complexidade do algoritmo é   ${O(t^2 + t \cdot C_U)}$, em que $C_U$ é custo de implementar $U$

## Simulação do algorítimo de Estimação de Fase Quântica

Para simular o algorítimo QPE usaremos a línguagem Ket de computação quântica, para isso precisamos ter ela instalada, caso não possua o pacote instalado rode o seguinte código:

```python
pip install ket-lang
```

Com a biblioteca instalada, importa-se para ser usada dentro do seu código:

In [3]:
from ket import *
from math import pi

A *Estimação de Fase Quântica (QPE)* utiliza a $IQFT$, portanto, é necessário a implementação da *Transformada Quântica de Fourier (QFT)*, que já foi explicada anteriormente.

In [4]:
def qft(qubits, do_swap: bool = True):
    if len(qubits) == 1:
        H(qubits)
    else:
        *init, last = qubits
        H(last)

        for i, ctrl_qubit in enumerate(reversed(init)):
            with control(ctrl_qubit):
                PHASE(pi / 2 ** (i + 1), last)

        qft(init, do_swap=False)

    if do_swap:
        size = len(qubits)
        for i in range(size // 2):
            SWAP(qubits[i], qubits[size - i - 1])

Uma vez que a implementação da *Transformada Quântica de Fourier* foi feita, pode-se implementar o algorítmo de *Estimação de Fase Quântica*

In [6]:
def estimador_de_fase(oraculo, precisão: int) -> float:
    assert precisão <= 20, "o tempo de computação pode ser muito grande"
    p = Process(simulator="dense", num_qubits=precisão + 1)
    ctr = H(p.alloc(precisão))
    tgr = X(p.alloc())
    for i, c in enumerate(ctr):
        ctrl(c, oraculo)(i, tgr)

    adj(qft)(ctr)  # <- chada da transformada de Fourier quântica inversa

    return measure(reversed(ctr)).value / 2**precisão


fase = pi

estimador_de_fase(
    oraculo=lambda i, tgr: PHASE(2 * pi * (fase / 10) * 2**i, tgr),
    precisão=18,
) * 10

6.858406066894531